In [40]:
reset -fs

In [1]:
%pylab inline
import networkx as nx
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='chaudao19', api_key='2eWbR48pGPcS4zVN0Ms2')
import plotly.plotly as py
from plotly.graph_objs import *
import pandas as pd
import numpy as np
import itertools
import pickle

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv("techtopics.csv")
df.columns = ["index","topics"]
# filter_words = [u'honda-gold-wing-riders',u'museums-galleries',u'thinkorswim',u'simple-homes',u'outdoors',u'midi-production',u'audio',u'mindfulness-meditation',u'art-museums',u'tao',u'parents',u'human-sexuality',u'intimacy',u'oculus-rift',"u'learn-how-to-sew'","u'spiritual-awakening'","u'book-swap'","u'drums'","u'board-games'","u'baking'","u'organic-foods'","u'real-estate-investment-education'","u'book-writing'","u'soccer'", "u'soccer'", "u'plays'", "u'worldreligions'", "u'humanrights'", "u'freethinker'", "u'running'", "u'worldreligions'", "u'romance-novels'", "u'conversazione-in-italiano'", "u'interfaith-and-intercultural-dialogue'", "u'food-waste'", "u'heathenry-or-heathenism-or-heathens'", "u'owasp'", "u'pagan-fellowship'", "u'housing'", "u'music'", "u'cooking'", "u'bible'", "u'health-and-alternative-health'", "u'whitewolf'", "u'whitewolf'", "u'm2m'", "u'nps'", "u'alpine-climbing'", "u'alpine-climbing'", "u'scrum'", "u'weightloss'", "u'social'", "u'fitness'", "u'fun-times'", "u'music-appreciation'"]

In [3]:
df.head()

,index,topics
0,0,"[u'Web Standards', u'PHP', u'Open Source', u'W..."
1,1,"[u'Social Software', u'New Technology', u'Web ..."
2,2,"[u'PHP', u'Open Source', u'Web Design', u'MySQ..."
3,3,[u'MySQL']
4,4,"[u'Enterprise Architecture', u'JavaScript', u'..."


In [4]:
type(df['topics'][0])

str

In [5]:
def clean_df(df):
    indice = df['topics'].apply(lambda x: type(x) == str)
    df = df[indice]
    df['topics'] = df['topics'].apply(lambda x: x[1:-1].split(','))
    df['topics'] = df['topics'].apply(lambda x: [word.strip(" ").lower() for word in x])
#     df['TF'] = df['key'].apply(lambda x: [True if word in filter_words else False for word in x])
#     indices = df['TF'].apply(lambda x: True not in x)
#     df = df[indices]
    df = df.reset_index()
    df = pd.DataFrame(df['topics'])
    return df

In [6]:
df = clean_df(df);

In [7]:
df.head(5)

,topics
0,"[u'web standards', u'php', u'open source', u'w..."
1,"[u'social software', u'new technology', u'web ..."
2,"[u'php', u'open source', u'web design', u'mysq..."
3,[u'mysql']
4,"[u'enterprise architecture', u'javascript', u'..."


In [8]:
def key_list(df):
    urlkey_unique = df['topics'].tolist()
    merged_list = list(itertools.chain(*urlkey_unique))
    return merged_list
def gettuples(df,column):
    urlkey_unique = df[column].tolist()
    merged_list = list(itertools.chain(*urlkey_unique))
    return merged_list

In [9]:
df['edges'] = df['topics'].apply(lambda x: [(a,b) for a in x for b in x if a != b])

In [10]:
# df.head(5)

In [11]:
G= nx.Graph()
nodes = list(set(key_list(df)))
G.add_nodes_from(nodes)
edges = gettuples(df,"edges")
# edges = [(a,b) for a in techtopics for b in techtopics if a != b]
G.add_edges_from(edges)

In [12]:
# plt.figure(figsize=(40,30))
# nx.draw(G,node_color='c',edge_color='k', with_labels=True);

In [13]:
pos= nx.spring_layout(G)
G.node = pos

In [14]:
dmin=1
ncenter=0
for n in pos:
    x,y=pos[n]
    d=(x-0.5)**2+(y-0.5)**2
    if d<dmin:
        ncenter=n
        dmin=d
        
p=nx.single_source_shortest_path_length(G,ncenter)

In [15]:
edge_trace = Scatter(
    x=[], 
    y=[], 
    line=Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')
for edge in G.edges():
    x0, y0 = G.node[edge[0]]
    x1, y1 = G.node[edge[1]]
    edge_trace['x'] += [x0, x1, None]
    edge_trace['y'] += [y0, y1, None]


In [16]:
node_trace = Scatter(
    x=[], 
    y=[], 
    text=[],
    mode='markers', 
    hoverinfo='text',
    marker=Marker(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='YIGnBu',
        reversescale=True,
        color=[], 
        size=10,         
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in G.nodes():
    x, y = G.node[node]
    node_trace['x'].append(x)
    node_trace['y'].append(y)
    node_trace['text'].append(node)

In [17]:
for node, adjacencies in enumerate(G.adjacency_list()):
    node_trace['marker']['color'].append(len(adjacencies))
    node_info = '# of connections: '+str(len(adjacencies))
    node_trace['text'].append(node_info)
    

In [18]:
fig = Figure(data=Data([edge_trace, node_trace]),
             layout=Layout(
                title='<br>Network Graph of Tech Topics ',
                titlefont=dict(size=20),
                showlegend=True, 
                width=800,
                height=650,
                hovermode='closest',
                margin=dict(b=20,l=5,r=1,t=40),
                annotations=[ dict(
                    text="Meetup",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=True),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=True)))

py.iplot(fig, filename='networkx')